# Ερώτηση 3 (Clustering)

In [13]:
import pandas as pd
import numpy as np
import json
import time 

yelp_dataset_path = '../yelp-dataset/'
buisness_v_path = yelp_dataset_path + 'yelp_academic_dataset_business.json'
leftK_path = '/home/left/Desktop/dataMining/set2/ask2/yelp_academic_dataset_business.json'

dataset_business = []
categories_order = ["Beauty & Spas", "Shopping", "Bars"]

start = time.time()

with open(buisness_v_path, 'r', encoding="utf8") as buisness_file:
    for line in buisness_file:
        json_dict = json.loads(line)
        if  json_dict['city'] == 'Toronto' and \
            json_dict['review_count'] > 10 and \
            json_dict['categories'] is not None and \
            any(word in json_dict['categories'] for word in categories_order):
                temp_cat = [cat for cat in categories_order if cat in json_dict['categories']][0]
                dataset_business.append([json_dict['business_id'], temp_cat])
                

stop = time.time()
print("Read file in {} seconds".format(stop-start))

Read file in 2.176222801208496 seconds


In [2]:
np_business = np.array(dataset_business)

len(np_business)

3265

In [3]:
def get_index(business_id):
    index = np.where(np_business == business_id)[0]
    return index

In [4]:
start = time.time()
business_review_txt = [[] for i in range(int(len(np_business)))]
review_v_path = yelp_dataset_path + 'yelp_academic_dataset_review.json'
leftK_path = '/home/left/Desktop/dataMining/set2/ask2/yelp_academic_dataset_review.json'

with open(review_v_path, 'r', encoding="utf8") as reviews_file:
    for line in reviews_file:
        json_dict = json.loads(line)
        index = get_index(json_dict['business_id'])

        if index.size > 0:
            if len(business_review_txt[index[0]]) == 0:
                business_review_txt[index[0]] = json_dict['text']
            else:
                business_review_txt[index[0]] += json_dict['text']

stop = time.time()

print("Read file in {} mins".format((stop-start)/60))

Read file in 4.731421732902527 mins


## TF-IDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
text = business_review_txt[23]
# just send in all your docs here
vectorizer = TfidfVectorizer()
fitted_vectorizer=vectorizer.fit([text])
tfidf_vectorizer_vectors=fitted_vectorizer.transform([text])
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0] 

values = np.array(first_vector_tfidfvectorizer.todense())[0]
labels = np.array(vectorizer.get_feature_names())
dict_labels = {}
for a, b in zip(labels, values):
    dict_labels[a] = b

#dict_labels